# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD003 - Aprendizado Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 04** - **Tópico: Máquinas de Vetores de Suporte**

<br>

O algoritmo **Máquinas de Vetores de Suporte**, conhecido por **SVM** (de *Support Vector Machines*, em inglês), é amplamente utilizado para tarefas de classificação e regressão, possuindo uma grande versatilidade e alto poder de modelagem para problemas lineares e não-lineares.

Conforme discutido em aula, o algoritmo encontra a fronteira de decisão formulando a tarefa de aprendizado como um problema de otimização. O SVM localiza a reta ou hiperplano que separa as instâncias de treinamento em duas regiões de forma a maximizar a margem entre os vetores de suporte das duas classes analisadas. Para problemas multiclasse, a tarefa de classificação é decomposta em um conjunto de problemas de classificação binária.

A versatilidade do SVM vem do uso de diferentes tipos de kernel, que são funções que realizam transformações nos dados a fim de projetá-los em um espaço de dimensão superior, de forma a permitir que relações não-lineares sejam modeladas com um classificador linear. Além do kernel linear, outros tipos de kernel muito utilizados são o Polinomial e o Radial Basis Function (RBF).

<br> 

**Objetivo deste notebook**: Exercitar o uso de SVM em tarefas de classificação, comparando diferentes tipos de kernel e realizando a otimização de hiperparâmetros do algoritmo.

<br>

---



##**Determinando a qualidade de vinhos verdes Portugueses**

*Seria possível prever as preferências humanas sobre vinhos verdes (principalmente de provadores profissionais) a partir de testes analíticos que quantificam propriedades químicas dos vinhos?*

Nesta atividade, vamos analisar um conjunto de observações sobre diversas amostras de vinho verde Português (do tipo branco), envolvendo suas propriedades químicas e classificação realizada por provadores através de análise sensorial. 

O preço do vinho depende de um conceito bastante abstrato de apreciação do vinho a partir de análise sensorial pelos provadores, cuja opinião pode ter um alto grau de variabilidade. Outro fator chave na certificação e avaliação da qualidade do vinho são os testes análiticos realizados em laboratório e facilmente disponíveis na etapa de certificação. Estes testes levam em consideração fatores como acidez, nível de pH, presença de açúcar e outras propriedades químicas. Para o setor vitivinícola, seria de interesse que a análise de qualidade humana no processo de degustação pudesse estar relacionada com as propriedades químicas do vinho aferidas com os testes analíticos. Assim,  o processo de avaliação e certificação da garantia da qualidade do vinho poderia ser mais controlado.


O conjunto de dados a ser analisado nesta atividade possui informações para 4898 amostras de vinhos verdes (tipo branco), todas produzidos em uma determinada zona de Portugal. Os atributos são coletados para 12 propriedades diferentes dos vinho: uma das quais é Qualidade, com base em análise sensorial por provadores, e as restantes são propriedades químicas dos vinhos, incluindo densidade, acidez, teor alcoólico, etc. Todas as propriedades químicas dos vinhos são variáveis ​​contínuas. A qualidade é uma variável ordinal com uma classificação possível de 1 (pior) a 10 (melhor). Cada variedade de vinho é provada por três provadores independentes e a classificação final atribuída é a classificação mediana dada pelos provadores.

O conjunto de dados original foi publicado no artigo: *Cortez, P., Cerdeira, A., Almeida, F., Matos, T., & Reis, J. (2009). Modeling wine preferences by data mining from physicochemical properties. Decision support systems, 47(4), 547-553.*

O objetivo desta tarefa é abordar a questão apontada no início desta seção, desenvolvendo um modelo capaz de predizer a classificação de um vinho com base nas suas propriedades químicas avaliadas em testes analíticos.



---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas necessárias
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais
from sklearn.svm import SVC  ## para treinar um SVM
from sklearn.model_selection import train_test_split # para divisão de dados
from sklearn.metrics import confusion_matrix, recall_score, precision_score,accuracy_score,ConfusionMatrixDisplay ## para avaliação dos modelos
sns.set()

In [ ]:
df = pd.read_table("https://drive.google.com/uc?export=view&id=11YsVJck74_gyADzGJSU9Uwn8cDq_l3BD",sep=";")
df.head()  # para visualizar apenas as 5 primeiras linhas

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'quality'* contém a classificação de qualidade de cada vinho, aferida pelos provadores. Vamos avaliar como as instâncias estão distribuídas entre os diferentes valores de *'quality'* presentes no dataset (originalmente os mesmos podem variar entre 0 e 10).

In [ ]:
## Distribuição do atributo alvo, 'quality'
sns.countplot(x='quality', data=df)

In [ ]:
## Imprimindo o valor exato de número de instâncias por nota
print(df.groupby('quality').size())

Podemos perceber que todos os vinhos no conjunto de dados estão classificados com notas entre 3 e 9. Entretanto, poucos vinhos estão classificados nos extremos da distribuição (notas 3 e 9). 

Também é importante sempre confirmarmos os tipos de dados de cada atributo, bem como se existem valores faltantes. A célula seguinte faz esta inspeção.

In [ ]:
df.info()

## Para analisar valores faltantes, quando codificados como NaN, podemos usar o
## comando abaixo
df.isnull().sum()

Vamos analisar a distribuição de valores para os atributos, os quais são todos numéricos como foi possível confirmar na célula de código anterior.

In [ ]:
## Criando vetor com nome dos atributos
features_names = df.columns.drop(['quality'])

In [ ]:
## Gerar um gráfico para cada variável numérica com a distribuição 
## de frequência. Avaliar a distribuição geral ou, opcionalmente, 
## a distribuição por classe (classificação do vinho) 

## Distribuição geral
def dist_plot(df,columns):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 3, indx+1)
        g = sns.histplot(x=var, data=df)
    plt.tight_layout()

## Distribuição por classe
def dist_plot_perclass(df,columns,label):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 3, indx+1)
        sns.color_palette("pastel")
        g = sns.histplot(x=var, data=df, hue=label,palette='muted')
    plt.tight_layout()


dist_plot(df, features_names)
dist_plot_perclass(df, features_names, 'quality')

**>>> Responda:** Neste ponto já é possível avaliar a necessidade de normalização dos dados para treinamento do SVM. Analise o quanto os dados demandam pré-processamento para normalização dos valores.

> ***Sua resposta aqui:*** Os atributos apresentam uma escala de valores muito diferentes. Por exemplo , o atributo "total sulfur dioxide" apresenta uma escala de valores muito alta. Enquanto os atributos "volatile acidity", "citric acid" e "chlorides", uma escala de valores muito baixa.

O gráfico abaixo mostra de outra forma como variam os valores dos atributos, considerando todas as instâncias no conjunto de dados.

**>>> Responda:** Quais atributos possuem as escalas de valores mais discrepantes em relação ao conjunto total de atributos?

> ***Sua resposta aqui:*** Os atributos free "sulfur dioxide" e "total sulfur dioxide" apresentam as escalas de valores mais discrepantes.

In [ ]:
df.drop(['quality'],axis=1).plot(figsize=(15,7))

Por fim, para auxiliar no entendimento do problema, é interessante avaliar a correlação entre atributos, bem como a correlação dos atributos com a classificação da qualidade do vinho. Vamos criar um heatmap com a correlação de Pearson entre os atributos da base.

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(df.corr(), annot=True, cmap = 'coolwarm')
plt.show()

É possível visualizar o comportamente da correlação entre atributos e a classe através de lineplots. Visualize os dados para alguns atributos selecionados que apresentem correlação positiva ou negativa com quality (mesmo que a correlação seja fraca).

In [ ]:
feature_sel = "alcohol" #@param {type:"string"}

plt.figure(figsize=(10,5))
sns.lineplot(data=df, x="quality",y=feature_sel,color='r')

---


### Estruturando a tarefa de classificação: vinho bom ou medíocre?

A análise exploratória dos dados nos mostrou que a qualidade dos vinhos no conjunto de dados disponível varia entre 3 e 9, com poucas instâncias sendo classificadas com as notas extremas. A distribuição dos dados sugere que a grande maioria dos vinhos possuem notas 5, 6 e 7. A falta de representatividade de vinhos com as demais classificações pode se apresentar como um problema para o desenvolvimento do modelo. 

**>>> Responda:** Qual possivelmente será o desempenho de um modelo treinado com estes dados para a classificação de novos vinhos com nota 9-10 ou nota 0-2?
 

> ***Sua resposta aqui:*** O modelo não teria um bom desempenho, pois falta representatividade dessa classe na nossa amostra.

Para dar continuidade à modelagem do problema de classificação dos vinhos, vamos **investigar a possibilidade de distinguir bons vinhos de vinhos medíocres ou ruins**, utilizando **a nota de 7** como um ponto de corte. Vinhos com nota 7 ou superior serão classificados como bons vinhos (1), enquanto vinhos com notas 0-6 serão classificados como medíocres (0).


In [ ]:
df['quality'] = df['quality'].replace([3, 4, 5, 6], 0)
df['quality'] = df['quality'].replace([7, 8, 9], 1)
sns.countplot(x='quality', data=df)


---


### Criando conjuntos de treino, validação e teste
 
Seguindo a mesma estratégia da aula passada, faremos a divisão dos dados em treino, validação e teste na proporção (sugerida) de 70%/15%/15% para fazer avaliação de desempenho dos modelos com um *holdout de 3 vias*. Na célula de código abaixo, os dados são inicialmente divididos entre duas variáveis: uma variável contendo os atributos e outra a classe a ser predita. Na sequência, implemente os seguintes passos:


1.   Utilize a fução train_test_split() para dividir os dados em treino, validação e teste. Caso tenha dúvidas, use como exemplo o código da aula 02.
2.   Normalize os dados usando o método MaxMin. Lembre-se que primeiramente os parâmetros de normalização são estimados a partir dos dados de treino e então aplicados a todos os três conjuntos.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['quality'], axis=1)
y = df['quality'].values

In [ ]:
## Definindo as proporções de treino, validação e teste.
train_ratio = 0.70
test_ratio = 0.15
validation_ratio = 0.15


## Fazendo a primeira divisão, para separar um conjunto de teste dos demais.
## Assuma X_train e y_train para os dados de treinamento e X_test e y_test para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos

### 
# Adicione aqui a sua resposta
##
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, stratify=y, random_state=42)

## Fazendo a segunda divisão, para gerar o conjunto de treino e validação a partir 
## do conjunto de 'treinamento' da divisão anterior
## Assuma X_train e y_train para os dados de treinamento e X_valid e y_valid para os de teste
## Dica: configure o random_state para facilitar reprodutibilidade dos experimentos

### 
# Adicione aqui a sua resposta
##
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio), stratify=y_train, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler 
## O MinMaxScaler transformará os dados para que fiquem no intervalo [0,1]
scaler = MinMaxScaler()

## Iniciar a normalização dos dados. Primeiro fazer um 'fit' do scaler nos  
## dados de treino. Esta etapa visa "aprender" os parâmetros para normalização.

### 
# Adicione aqui a sua resposta
##
scaler.fit(X_train)

## Aplicar a normalização nos três conjuntos de dados:

### 
# Adicione aqui a sua resposta
##

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)


Com os dados normalizados, podemos melhor inspecionar a distribuição de valores por classe, considerando o problema de classificação binário. Vamos fazer a análise para o conjunto de treino, observando as medianas para cada classe.

In [ ]:
df_train_norm =  pd.DataFrame(X_train,columns=features_names)
df_ytrain =  pd.DataFrame(y_train,columns=['quality'],dtype=int)
df_train_norm =  pd.concat([df_train_norm,df_ytrain], axis=1)
ave_values = df_train_norm.groupby("quality").median()
ave_values.plot(kind="bar",figsize=(15,7))

---


### Treinamento de modelos SVM com Kernel Linear



Como vimos em aula, o modelo SVM com Kernel Linear é o "mais simples" dentre os modelos de SVM. Ele possui o hiperparâmetro C (termo de regularização), comum a todos os modelos de SVM, e que precisa ser otimizado para cada problema.

#### Otimização "manual" do hiperparâmetro C
No código abaixo, vamos fazer a otimização do hiperparâmetro C através da implementação de um loop. Para cada modelo, vamos avaliar a acurácia, recall e precisão. Vamos assumir que o melhor modelo será aquele que maximiza a acurácia.

In [ ]:
## Definindo um array para armazenar o desempenho de cada modelo treinado e avaliado
perf_valid = []

## Definindo valores de C a serem testados
param_grid_C = [0.1, 1, 5, 10, 50, 100] #valores para C, termo de regularização

# Treinando e avaliado os modelos com cada valor de hiperparâmetro especificado
for ii in range(len(param_grid_C)):
    #clf = SVC(kernel='linear',C=param_grid_C[ii],random_state=42)#, class_weight='balanced') ##class_weight minimiza o efeito de termos mais exemplos para vinhos medíocres/ruins
    
    # clf = SVC(kernel='linear', C=param_grid_C[ii], random_state=42)
    clf = SVC(kernel='linear',C=param_grid_C[ii], class_weight='balanced',random_state=42)

    clf.fit(X_train, y_train)
    pred_i = clf.predict(X_valid)
    perf_valid.append([param_grid_C[ii],accuracy_score(y_valid, pred_i),recall_score(y_valid, pred_i),precision_score(y_valid, pred_i)])

In [ ]:
perf_df = pd.DataFrame(perf_valid, columns=['C','accuracy','recall','precision'])
perf_df

In [ ]:
plt.figure(figsize=(12, 6))
## Transforma o dataframe para facilitar plotar todas as métricas na mesma figura
perf_df_melt = pd.melt(perf_df, id_vars=['C'], value_vars=['accuracy','recall','precision'])
sns.lineplot(data=perf_df_melt,x='C',y='value',hue='variable',palette='muted',marker='o')

**>>> Responda:** Qual valor de C maximiza a acurácia? Como são os valores de acurácia, recall e precisão para este modelo?

> ***Sua resposta aqui:*** Quando C é 10 a acurácia foi 0.732, variando pouco a partir disso. O valor de recall foi 0.855 e a precisão 0.439.

**>>> Responda:** Qual o impacto de usar a opção `class_weight='balanced'` no código?

> ***Sua resposta aqui:*** através dessa opção foi possível encontrar os valores de precisão do modelo.

#### Otimização do hiperparâmetro C com GridSearchCV

O scikit-learn disponibiliza o método GridSearchCV, que permite fazer a otimização de hiperparâmetros de forma mais automática. É especialmente útil quando queremos testar c**ombinações** de valores para diferentes hiperparâmetros. O código abaixo aplica o GridSearchCV usando as divisões de dados de treino e validação pré-definidas (através do método PredefinedSplit). Portanto, devemos chegar ao mesmo resultado da análise anterior. 

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV ## para auxiliar na otimização de hiperparâmetros

# Cria lista com os dados de treinamento com índice -1 e dados de validação com índice 0
# Concatena os dados de treino e validação com as partições pré-definidas
split_index = [-1]*len(X_train) + [0]*len(X_valid)
X_gridSearch = np.concatenate((X_train, X_valid), axis=0)
y_gridSearch = np.concatenate((y_train, y_valid), axis=0)
pds = PredefinedSplit(test_fold = split_index)

## Define métricas de desempenho a serem estimadas
scoring = {'Accuracy':'accuracy', 'Precision': 'precision', 'Recall':'recall'} 

## Define o algoritmo base da otimização de hiperparâmetros
estimator = SVC(kernel='linear',class_weight='balanced')

## Define a grid de hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1, 5, 10, 50, 100]}#, 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}

## Aplica GridSearch com as partições de treino/validação pré-definidas
gridS = GridSearchCV(estimator = estimator,
                   cv=pds,
                   param_grid=param_grid, 
                   scoring=scoring,
                   refit='Accuracy', ##métrica a ser utilizada para definir o melhor modelo, retreinando-o com toda a base
                   return_train_score=True)
gridS.fit(X_gridSearch, y_gridSearch)
print('Desempenho máximo obtido com: {}'.format(gridS.best_params_))

A visualização dos dados pode nos auxiliar a explorar o resultado da otimização de hiperparâmetros. O código abaixo criar um gráfico a partir dos resultados do GridSearchSV, focando na variação de um hiperparâmetro. Neste caso, analisamos C, o único hiperparâmetro variado na análise. (*OBS.: Não se preocupe se não entender todos os detalhes de implementação da função, ela é apenas um utilitário na visualização de dados*)

In [ ]:
## O código desta célula cria um gráfico de variação de desempenho de acordo com
## o valor do hiperparâmetro C.

results = gridS.cv_results_

plt.figure(figsize=(10, 7))
plt.title("Resultados do GridSearchCV",
      fontsize=16)

plt.xlabel("Hyperparameter") ##nome do parâmetro a ser analisado
plt.ylabel("Performance")

ax = plt.gca()

## Criar um numpy array para os resultados do hiperparâmetro a ser analisado.
## O hiperparâmetro C está identificado no objeto retornado pelo gridSearchCV
## como param_C
X_axis = np.array(results['param_C'].data, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k', 'b', 'r']):
    for sample, style in (('train', '--'), ('test', '-')):
       sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
       sample_score_std = results['std_%s_%s' % (sample, scorer)]
       ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                    sample_score_mean + sample_score_std,
                    alpha=0.1 if sample == 'test' else 0, color=color)
       ax.plot(X_axis, sample_score_mean, style, color=color,
            alpha=1 if sample == 'test' else 0.7,
            label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    ## Plota uma linha vertical para o valor de hiperparâmetro que maximiza a métrica de desempenho
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
        linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)
    ## Anota o valor do melhor score
    ax.annotate("%0.3f" % best_score,
            (X_axis[best_index], best_score + 0.008))

plt.legend(loc="best")
plt.grid(False)
plt.show()

#### Visualização da fronteira de decisão de um SVM Linear

Abaixo definimos uma funçao auxiliar para visualizar a fronteira de decisão dos classificadores, semelhante à que foi usada em outras atividades práticas.

In [ ]:
## Fonte: https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Utilities/ML-Python-utils.py
def plot_decision_boundaries(X, y, limit, model_class, **model_params):
    """
    Function to plot the decision boundaries of a classification model.
    This uses just the first two columns of the data for fitting 
    the model as we need to find the predicted value for every point in 
    scatter plot.
    Arguments:
            X: Feature data as a NumPy-type array.
            y: Label data as a NumPy-type array.
            model_class: A Scikit-learn ML estimator class 
            e.g. GaussianNB (imported from sklearn.naive_bayes) or
            LogisticRegression (imported from sklearn.linear_model)
            **model_params: Model parameters to be passed on to the ML estimator
    
    Typical code example:
            plt.figure()
            plt.title("KNN decision boundary with neighbros: 5",fontsize=16)
            plot_decision_boundaries(X_train,y_train,KNeighborsClassifier,n_neighbors=5)
            plt.show()
    """
    try:
        X = np.array(X)
        y = np.array(y).flatten()
    except:
        print("Coercing input data to NumPy arrays failed")

    # Reduces to the first two columns of data - for a 2D plot!
    reduced_data = X[:, :2]
    # Instantiate the model object
    model = model_class(**model_params)
    # Fits the model with the reduced data
    model.fit(reduced_data, y)

    # Step size of the mesh. Decrease to increase the quality of the VQ.
    h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].    

    # Plot the decision boundary. For that, we will assign a color to each
    x_min, x_max = reduced_data[:, 0].min() - limit, reduced_data[:, 0].max() + limit
    y_min, y_max = reduced_data[:, 1].min() - limit, reduced_data[:, 1].max() + limit
    # Meshgrid creation
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

    # Obtain labels for each point in mesh using the model.
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])    

    x_min, x_max = X[:, 0].min() - limit, X[:, 0].max() + limit
    y_min, y_max = X[:, 1].min() - limit, X[:, 1].max() + limit
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))

    # Predictions to obtain the classification results
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

    # Plotting
    plt.contourf(xx, yy, Z, alpha=0.2,cmap='viridis')
    g=plt.scatter(X[:, 0], X[:, 1], c=y, alpha=0.6,s=50, edgecolor='k', cmap='viridis' )
    plt.xlabel("Feature-1",fontsize=15)
    plt.ylabel("Feature-2",fontsize=15)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.legend(handles=g.legend_elements()[0],labels=('0','1'))
    return plt

Visualizamos abaixo a fronteira de decisão para o modelo SVM com Kernel Linear. É importante atentar para o fato de que este exemplo visual se baseia apenas em dois atributos para manter o problema em duas dimensões, e assim facilitar a visualização. Assim, este modelo **não representa** exatamente o modelo treinado acima, é apenas um exemplo de como a fronteira de decisão deste modelo SVM Linear poderia se comportar nos dados. Além disso, para melhorar a visualização usamos apenas as primeiras 100 instâncias do conjunto de dados.

In [ ]:
## Visualizar a fronteira de decisão para o modelo SVM com kernel linear. 
## >>> Ajuste o valor de C para o melhor valor encontrado o GridSearchCV (1 é o valor padrão)
plot_decision_boundaries(X_train[1:100,], y_train[1:100], 0.1, SVC, kernel='linear',class_weight='balanced',C=10)

**>>> Exercício:** Com base no melhor valor do hiperparâmetro C encontrado na análise do GridSearch para o SVM Linear, aplique o modelo para classificação nos dados de teste (X_test), avaliando o seu desempenho com as métricas de acurácia, recall e precisão (opcionalmente, avalie também a matriz de confusão).

OBS.: O método `GridSearchCV` com opção `refit=True` ou `refit='Accuracy'` (ou outra métrica de desempenho) realiza o treinamento de um modelo com toda a base informada na função `.fit()`. No nosso caso, se refere aos dados de treino e validação.

In [ ]:
### 
# Adicione aqui a sua resposta
##

# Opção 1: rodando novamente SVC com C ótimo = 10
print("Opção 1")
clf_10 = SVC(kernel='linear',C=10, class_weight='balanced',random_state=42)
clf_10.fit(X_train, y_train)

pred_10 = gridS.predict(X_test)

print(accuracy_score(y_test, pred_10))
print(recall_score(y_test, pred_10))
print(precision_score(y_test, pred_10))

cm = confusion_matrix(y_test, pred_10 ,labels=clf_10.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_10.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

# Opção 2: utilizando o pŕoprio modelo  ótimo gridS (GridSearchCV)
print("Opção 2")
pred_10_gridS = gridS.predict(X_test)

print(accuracy_score(y_test, pred_10_gridS))
print(recall_score(y_test, pred_10_gridS))
print(precision_score(y_test, pred_10_gridS))

cm = confusion_matrix(y_test, pred_10_gridS ,labels=gridS.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gridS.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

---


### Comparando diferentes funções de Kernel no SVM


Nesta seção, vamos comparar diferentes tipos de Kernel em um algoritmo SVM. Utilizaremos o método GridSearchCV do scikit-learn pela praticidade do mesmo. Entretanto, também seria possível realizar a mesma análise com um loop definido por nós, testando diferentes combinações de valores de hiperparâmetros.

In [ ]:
from sklearn.model_selection import PredefinedSplit, GridSearchCV ## para auxiliar na otimização de hiperparâmetros


## Cria lista com os dados de treinamento com índice -1 e dados de validação com índice 0
## Concatena os dados de treino e validação com as partições pré-definidas
split_index = [-1]*len(X_train) + [0]*len(X_valid)
X_gridSearch = np.concatenate((X_train, X_valid), axis=0)
y_gridSearch = np.concatenate((y_train, y_valid), axis=0)
pds = PredefinedSplit(test_fold = split_index)

# ## Define métricas de desempenho a serem estimadas
scoring = {'Accuracy':'accuracy', 'Precision': 'precision', 'Recall':'recall'} 

## Define o algoritmo base da otimização de hiperparâmetros
estimator = SVC(class_weight='balanced')

## Define a grid de hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1, 5, 10, 50, 100], 'kernel': ['rbf', 'poly', 'sigmoid']}#,'gamma': [1,0.1,0.01,0.001]} ## gamma foi removido para reduzir tempo de execução

## Aplica GridSearch com as partições de treino/validação pré-definidas
gridS2 = GridSearchCV(estimator = estimator,
                   cv=pds,
                   param_grid=param_grid, 
                   scoring=scoring,
                   refit='Accuracy', ##métrica a ser utilizada para definir o melhor modelo, retreinando-o com toda a base
                   return_train_score=True)
gridS2.fit(X_gridSearch, y_gridSearch)
print('Desempenho máximo obtido com: {}'.format(gridS2.best_params_))

In [ ]:
y_pred_cv = gridS2.predict(X_test)
cm = confusion_matrix(y_test, y_pred_cv ,labels=gridS2.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gridS2.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

In [ ]:
results = gridS2.cv_results_

plt.figure(figsize=(10, 7))
plt.title("Resultados do GridSearchCV",
      fontsize=16)

plt.xlabel("Hyperparameter") ##nome do parâmetro a ser analisado
plt.ylabel("Performance")

ax = plt.gca()

## Criar um numpy array para os resultados do hiperparâmetro a ser analizado
X_axis = np.array(results['param_kernel'].data)#, dtype=float)

for scorer, color in zip(sorted(scoring), ['g', 'k', 'b', 'r']):
    for sample, style in (('train', '--'), ('test', '-')):
       sample_score_mean = results['mean_%s_%s' % (sample, scorer)]
       sample_score_std = results['std_%s_%s' % (sample, scorer)]
       ax.fill_between(X_axis, sample_score_mean - sample_score_std,
                    sample_score_mean + sample_score_std,
                    alpha=0.1 if sample == 'test' else 0, color=color)
       ax.plot(X_axis, sample_score_mean, style, color=color,
            alpha=1 if sample == 'test' else 0.7,
            label="%s (%s)" % (scorer, sample))

    best_index = np.nonzero(results['rank_test_%s' % scorer] == 1)[0][0]
    best_score = results['mean_test_%s' % scorer][best_index]

    ## Plota uma linha vertical para o valor de hiperparâmetro que maximiza a métrica de desempenho
    ax.plot([X_axis[best_index], ] * 2, [0, best_score],
        linestyle='-.', color=color, marker='x', markeredgewidth=3, ms=8)
    ## Anota o valor do melhor score
    ax.annotate("%0.3f" % best_score,
            (X_axis[best_index], best_score + 0.008))

plt.legend(loc="best")
plt.grid(False)
plt.show()

**>>> Responda:** Qual foi a melhor configuração de hiperparâmetros encontrada pelo GridSearchCV? Qual o respectivo desempenho?

> ***Sua resposta aqui:*** Para precisão e acurácia os melhores valores foram encontrados com o hiperparâmetro poly, precision = 0.460 e accuracy = 0.752. E o melhor valor de recall foi encontrado para o hiperperâmetro rfb, recall = 0.881.

A fim de entender como a fronteira de decisão muda para um kernel RBF ou Polinomial em relação ao kernel Linear, vamos visualizá-la para uma amostra dos dados.

In [ ]:
## Visualizar a fronteira de decisão para o melhor modelo SVM
## Ajuste o valor de C e gamma para o melhor valor encontrado o GridSearchCV (1 e 'auto' são valores padrões)
#plot_decision_boundaries(X_train[1:100,], y_train[1:100], 0.1, SVC, kernel='rbf',class_weight='balanced',C=1,gamma='auto')
plot_decision_boundaries(X_train[1:100,], y_train[1:100], 0.1, SVC, kernel='poly',class_weight='balanced',C=10,gamma='auto')

**Opcional:** Para o tipo de Kernel que apresentar o melhor desempenho na análise realizada, estenda o estudo de impacto de hiperparâmetros para otimizar o valor de gamma.

**>>> Exercício:** Avalie o desempenho do melhor modelo sobre os dados de teste. Compare o resultado com o desempenho obtido com o SVM Linear, discutindo se a acurácia, precisão e recall melhoraram/pioraram, bem como se o modelo foi capaz de aumentar TP/TN ou diminuir FP/FN.


In [ ]:
### 
# >>> Adicione aqui a sua resposta
##
# Método linear: gridS (GridSearchCV)
print("Modelo ótimo linear  {'C': 10}")
pred_10_gridS = gridS.predict(X_test)

print(accuracy_score(y_test, pred_10_gridS))
print(recall_score(y_test, pred_10_gridS))
print(precision_score(y_test, pred_10_gridS))

cm = confusion_matrix(y_test, pred_10_gridS ,labels=gridS.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gridS.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print("Modelo ótimo não linear {'C': 0.1, 'kernel': 'poly'}")
y_pred_cv = gridS2.predict(X_test)
print(accuracy_score(y_test, y_pred_cv))
print(recall_score(y_test, y_pred_cv))
print(precision_score(y_test, y_pred_cv))


y_pred_cv = gridS2.predict(X_test)
cm = confusion_matrix(y_test, y_pred_cv ,labels=gridS2.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gridS2.classes_)
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

print("Análise:")

print("Os valores de acurácia e precisão melhoraram, apesar do valor de recall piorar.")
print("O modelo linear se mostrou melhor na classificação dos VP, 115 contra 110 do modelo não linear.")
print("Enquanto o modelo não linear se mostrou melhor na classificação dos VN, 429 contra 405 do modelo linear.")


---

## Sua vez



Desenvolva um modelo preditivo interpretando o problema como uma tarefa de regressão, ao invés de uma classificação. Faça uso de um modelo SVM Regressor (SVR) e avalie o desempenho com as métricas **Mean Absolute Error** e **R2**, disponíveis no scikit-learn. Para fins de referência, os autores do artigo no qual os dados originais foram publicados desenvolveram um modelo SVM que obteve MAE de 0.45.

Faça também uma avaliação da acurácia do modelo seguindo estratégia similar a dos autores: considerando um nível de tolerância $T$, a saída predita pelo modelo $ŷ_i$ pode ser transformada em uma categoria $p_i$ (neste caso, notas inteiras entre 0 e 10) através do seguinte critério:
$ pi = y_i$, se $|y_i − ŷ_i| ≤ T$, senão $p_i = y_i′$, onde  $y_i′$ denota a classe mais próxima de $ŷ_i$. Por exemplo, supondo $ŷ_i = 4.9$ e $T = 0.25$, como $|5.0 - 4.9| = 0.1 ≤ 0.25$, esta instância seria categorizada como 5. No caso de $ŷ_i = 4.4$ e o mesmo nível de tolerância, como a comparação de $ŷ_i$ com os valores 4 e 5 não atendem ao critério de tolerância, a classe predita será a mais próxima de 4.4, ou seja, 4. Experimente para T=0.25 e, opcionalmente, para T=0.5.

Dicas para o exercício:


*   Utilize o método [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html?highlight=svr#sklearn.svm.SVR) do Scikit-learn (`from sklearn.svm import SVR`). O método tem os mesmos argumentos que foram utilizados para o caso da classificação. 
*   O modelo SVR pode ser otimizada com o uso de GridSearchCV e PredefinedSplits, semelhante ao que foi feito para o modelo de classificação binária neste notebook.
*   As métricas de avaliação de modelos sugeridas podem ser importadas através do comando `from sklearn.metrics import mean_absolute_error, r2_score`. Para avaliar o R2, por exemplo, utilize `r2_score(y_test, y_pred)`
*   Utilize um scatterplot para comparar os valores preditos com os valores reais.



> ***Sua vez:*** 

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score

# recarrega dados
df = pd.read_table("https://drive.google.com/uc?export=view&id=11YsVJck74_gyADzGJSU9Uwn8cDq_l3BD",sep=";")

## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['quality'], axis=1)
y = df['quality'].values

## Definindo as proporções de treino, validação e teste.
train_ratio = 0.70
test_ratio = 0.15
validation_ratio = 0.15

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, stratify=y, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio), stratify=y_train, random_state=42)

## Normaliza os dados
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

## Cria lista com os dados de treinamento com índice -1 e dados de validação com índice 0
## Concatena os dados de treino e validação com as partições pré-definidas
split_index = [-1]*len(X_train) + [0]*len(X_valid)
X_gridSearch = np.concatenate((X_train, X_valid), axis=0)
y_gridSearch = np.concatenate((y_train, y_valid), axis=0)
pds = PredefinedSplit(test_fold = split_index)

# ## Define métricas de desempenho a serem estimadas
scoring = {'MAE':'neg_mean_absolute_error', 'R2': 'r2'}

## Define o algoritmo base da otimização de hiperparâmetros
estimator = SVR()

## Define a grid de hiperparâmetros a serem testados
param_grid = {'C': [0.1, 1, 5, 10, 20], 'gamma': [1,0.1,0.01,0.001], 'kernel': ['rbf', 'poly', 'linear']} # muito lento com muitos parametros
#param_grid = {'C': [0.1, 1, 5, 10, 50, 100]}

## Aplica GridSearch com as partições de treino/validação pré-definidas
gridS3 = GridSearchCV(estimator = estimator,
                   cv=pds,
                   param_grid=param_grid, 
                   scoring=scoring,
                   refit='MAE', ##métrica a ser utilizada para definir o melhor modelo, retreinando-o com toda a base
                   return_train_score=True)
gridS3.fit(X_gridSearch, y_gridSearch)
print('Desempenho máximo obtido com: {}'.format(gridS3.best_params_))


In [ ]:
## testando predição: r2_score
y_pred_svr = gridS3.predict(X_test)
print(r2_score(y_test, y_pred_svr))

# plotando valores preditos com valores reais
print('Antes do agrupamento dos valores em classes')
plt.scatter(y_test, y_pred_svr, color = 'red')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.show()

# de-para de valores T=0.25
#print(y_pred_svr)
T = 0.25
y_pred_svr_T = []
faixas = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for y in y_pred_svr:
  # print('valor: ', format(y))
  atribui = -1 # valor a ser atribuido
  proximidade = 1 # grau proximidade
  for y_faixa in faixas:
    if abs(y_faixa - y) < proximidade:
      proximidade = abs(y_faixa - y) # atualiza grau proximidade
      y_proximo = y_faixa
    if abs(y_faixa - y) < T: # | y_pred - y | < T Então atribui y_pred
      atribui = y_faixa
      break
  if atribui == -1: # se não encontrou valor pelo T, atribui y_proximo
    atribui = y_proximo
  # print('valor: ', y, ' - atribui: ', atribui)
  y_pred_svr_T.append(atribui)

print('Depois do agrupamento dos valores em classes')
plt.scatter(y_test, y_pred_svr_T, color = 'red')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.show()

print('Matriz de confusão')
cm = confusion_matrix(y_test, y_pred_svr_T) # como visualizar os labels corretos?
disp = ConfusionMatrixDisplay(confusion_matrix=cm) # como visualizar os labels corretos?
disp = disp.plot(include_values=True, cmap='Blues', ax=None, xticks_rotation='horizontal')
plt.grid(False)
plt.show()

  




---




#### Sugestões de experimentos extras:

*  Explore o hiperparâmetro class_weights, tentando outros pesos e observando o impacto sobre o desempenho do modelo.
*   Tente aplicar o modelo desenvolvido ao conjunto de dados de [vinhos verdes tintos](https://drive.google.com/file/d/1wkvrdgqXRXrA9Tk2lV0K-cOjf4vSQh8U/view?usp=sharing), que possui as mesmas propriedades analisadas, entretanto, difere dos dados usados neste exercício por ser do tipo tinto. Aplique seu modelo de SVR (Regressor) ou o modelo de SVM para classificação binária desenvolvido nesta atividade. O desempenho preditivo para este novo conjunto de dados é tão bom quanto o obtido para os dados de teste (15% da base original)? Caso não seja bom, reflita sobre o possível motivo deste resultado.  Você acha que o desempenho poderia melhorar ao se desenvolver um modelo a partir da união das bases de vinhos verdes brancos e tintos? Para fazer download dos dados no notebook, use o seguinte comando: `df = pd.read_table("https://drive.google.com/uc?export=view&id=1wkvrdgqXRXrA9Tk2lV0K-cOjf4vSQh8U",sep=";")`.